In [ ]:
import os
import cv2
import numpy as np

In [ ]:
features = []
target = []
for i in list(range(0,43)):
    collecttionImagesName = os.listdir('../input/traffic-signs-classification/myData' + '/' + str(i))
    for j in collecttionImagesName:
        img = cv2.imread('../input/traffic-signs-classification/myData' + '/' + str(i) + '/' + str(j))
        features.append(img)
        target.append(i)
    print('Lodding Data in folder', i)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
training_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.2)

##### Image Augmentation: It is a techniques used to generate the images in different different angles

In [ ]:
# Normalization: Reducing the range of pixels by dividing it by 255

def preprocessing(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = image/255
    return image

In [ ]:
training_features = np.array(list(map(preprocessing, training_features)))

In [ ]:
test_features = np.array(list(map(preprocessing, test_features)))

In [ ]:
training_features.shape

In [ ]:
training_features = training_features.reshape(58511, 32, 32, 1)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
dataGen = ImageDataGenerator(rotation_range=10, width_shift_range= 0.1, height_shift_range=0.1, zoom_range=0.1, shear_range=0.1)

In [ ]:
dataGen.fit(training_features)

In [ ]:
batches = dataGen.flow(training_features, train_target, batch_size=20)

In [ ]:
len(batches)

In [ ]:
images, label = next(batches)

In [ ]:
images.shape

In [ ]:
images = images.reshape(20, 32, 32)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,10))
for i in range(20):
    plt.subplot(4,5,i+1)
    plt.imshow(images[i])
plt.show()

In [ ]:
from keras.utils import to_categorical

In [ ]:
train_target = to_categorical(train_target)

In [ ]:
train_target.shape

In [ ]:
training_features.shape

# Step 1: Define the Architecture 

In [ ]:
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Conv2D(60,(3,3),activation = 'relu', input_shape=(32,32,1)))
model.add(Conv2D(60,(3,3),activation = 'relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(30,(3,3),activation='relu'))
model.add(Conv2D(30,(3,3),activation='relu'))
model.add(Conv2D(30,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(43, activation='softmax'))

# Step 2: Compile

In [ ]:
from keras.optimizers import Adam

In [ ]:
model.compile(Adam(lr=0.001), loss = 'categorical_crossentropy', metrics=['accuracy'])

# Step 3: Train the Model

In [ ]:
 # It will generate new images, compile it with original images and the train on those combined images
model.fit_generator(dataGen.flow(training_features, train_target, batch_size=20), epochs = 20)

In [ ]:
ModelInJson = model.to_json()

In [ ]:
file = open('./Traffic_Signs_Model.json', 'w')
file.write(ModelInJson)
file.close()

In [ ]:
model.save_weights('./Traffic_Signs_Model.h5')

In [ ]:
from keras.models import model_from_json

In [ ]:
file=open('./Traffic_Signs_Model.json', 'r')
loaded_data = file.read()
loaded_model = model_from_json(loaded_data)
loaded_model.load_weights("./Traffic_Signs_Model.h5")

# Step 3: Test the model by using it for predictions

In [ ]:
test_features = np.array(list(map(preprocessing, test_features)))